In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

2025-09-15 09:22:03.092280: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
IMG_SIZE = (64, 64)

In [3]:
# -----------------------------
# 1. Build Autoencoder Model
# -----------------------------
def build_autoencoder():
    encoder = models.Sequential([
        layers.Input(shape=(64, 64, 3)),
        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
    ])

    decoder = models.Sequential([
        layers.Conv2DTranspose(64, (3, 3), strides=2, activation="relu", padding="same"),
        layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"),
        layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same"),  # output image
    ])

    autoencoder = models.Sequential([encoder, decoder])
    autoencoder.compile(optimizer="adam", loss="mse")
    return autoencoder

autoencoder = build_autoencoder()
autoencoder.summary()

I0000 00:00:1757928131.789111   48638 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3620 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 16, 16, 64)     │        19,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 64, 64, 3)      │        56,259 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,651 (295.51 KB)

 Trainable params: 75,651 (295.51 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# -----------------------------
# 2. Load Only "Engine" Images for Training
# -----------------------------
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    "dataset",              # your dataset folder with /Engine only
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode="input",     # input = output (autoencoder)
    subset="training"
)

val_gen = datagen.flow_from_directory(
    "dataset",
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode="input",
    subset="validation"
)

Found 22 images belonging to 2 classes.
Found 5 images belonging to 2 classes.


In [5]:
# -----------------------------
# 3. Train Autoencoder
# -----------------------------
autoencoder.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen
)

Epoch 1/20


/home/sachi/ml_project/assignment/env/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-09-15 09:22:20.304300: I external/local_xla/xla/service/service.cc:163] XLA service 0x74ebf4004130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-15 09:22:20.304341: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2025-09-15 09:22:20.338510: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-15 09:22:20.594226: I external/local_xla/xla/st

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - loss: 0.0819

I0000 00:00:1757928144.913908   48724 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - loss: 0.0819 - val_loss: 0.0714
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0814 - val_loss: 0.0714
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0811 - val_loss: 0.0714
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0806 - val_loss: 0.0715
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0799 - val_loss: 0.0717
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0791 - val_loss: 0.0721
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0781 - val_loss: 0.0729
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 0.0771 - val_loss: 0.0739
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0760 - val_loss: 0.0748
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0751 - val_loss: 0.0744
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.0738 - val_loss: 0.0726
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.0723 - val_loss: 0.0705
Epoch 13/20
1

In [14]:
# -----------------------------
# 4. Prediction Function
# -----------------------------
def predict_image(img_path, threshold=0.02):
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    reconstructed = autoencoder.predict(img_array)
    mse = np.mean(np.square(img_array - reconstructed))  # reconstruction error

    if mse < threshold:
        return "Engine", 1 - mse  # higher confidence if error is small
    else:
        return "Unknown", mse     # larger error → not engine

In [15]:
# -----------------------------
# 5. Example Test
# -----------------------------
pred, conf = predict_image("12309_14_2.jpg")
print(f"Predicted: {pred} (Confidence: {conf:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted: Unknown (Confidence: 0.0833)
